In [ ]:
import matplotlib, matplotlib.pyplot as plt, numpy as np
from astropy.io import fits 
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from matplotlib.colors import Normalize 

matplotlib.rcParams['animation.embed_limit'] = 100
plt.style.use('mplstyle.txt')
%matplotlib widget

# spectra animation

read in spectra

In [ ]:
hdul = fits.open('data/SMC_SNR/blah4.fits')
flux = hdul[0].data
if 'CD1_1' in list(hdul[0].header.keys()):
    wave = hdul[0].header['CRVAL1'] + hdul[0].header['CD1_1'] * (np.arange(hdul[0].header['NAXIS1']) - hdul[0].header['CRPIX1'] + 1)
else:
    wave = hdul[0].header['CRVAL1'] + hdul[0].header['CDELT1'] * (np.arange(hdul[0].header['NAXIS1']) - hdul[0].header['CRPIX1'] + 1)

function to animate

In [ ]:
def spectrum_ani(frame):
    ax.clear()
    ax.plot(wave,flux[frame], lw=0.6)
    ax.set_xlabel(r'$\rm Wavelength$'); ax.set_ylabel(r'$\rm Intensity$')
    ax.set_ylim(-670,2550); ax.text(0.02, 0.95, rf'$\rm Frame\ {frame}$', ha='left', va='top', transform=ax.transAxes, fontsize=12)
    return ax

In [ ]:
fig,ax=plt.subplots(1,1, figsize=(7,3.5), layout='compressed')
spec_ani = FuncAnimation(fig=fig, func=spectrum_ani, frames=len(flux), interval=75, blit=False, repeat=True, cache_frame_data=False)
plt.close(fig)
HTML(spec_ani.to_jshtml())

save the animation to a .gif file

In [ ]:
writer = matplotlib.animation.PillowWriter(fps=20)
spec_ani.save('spectrum_animation.gif', writer=writer)

# hex animation

read in spectra

In [ ]:
name = '0885.norm.dc.fits'
datdir = 'data/SMC_SNR/'

hdul = fits.open(datdir + 'b' + name)
bflux = hdul[0].data
if 'CD1_1' in list(hdul[0].header.keys()):
    bwave = hdul[0].header['CRVAL1'] + hdul[0].header['CD1_1'] * (np.arange(hdul[0].header['NAXIS1']) - hdul[0].header['CRPIX1'] + 1)
else:
    bwave = hdul[0].header['CRVAL1'] + hdul[0].header['CDELT1'] * (np.arange(hdul[0].header['NAXIS1']) - hdul[0].header['CRPIX1'] + 1)

hdul = fits.open(datdir + 'r' + name)
rflux = hdul[0].data
if 'CD1_1' in list(hdul[0].header.keys()):
    rwave = hdul[0].header['CRVAL1'] + hdul[0].header['CD1_1'] * (np.arange(hdul[0].header['NAXIS1']) - hdul[0].header['CRPIX1'] + 1)
else:
    rwave = hdul[0].header['CRVAL1'] + hdul[0].header['CDELT1'] * (np.arange(hdul[0].header['NAXIS1']) - hdul[0].header['CRPIX1'] + 1)

read in hex bin data, sort them to follow the snake-like pattern

In [ ]:
hex = np.loadtxt('data/fiber_info/STD Array Elements-Table 1.csv', delimiter=',', skiprows=13, usecols=(3,4), max_rows=552)
sorted_data = []  # sort the data in a snake-like pattern
for i, y in enumerate(np.unique(hex[:, 1])):  # get all vertical values
    row_mask = hex[:, 1] == y
    row_data = hex[row_mask]
    sorted_indices = np.argsort(row_data[:, 0])  # sort by x coord
    if i % 2 == 0:  # even index rows go right-to-left, odd go left-to-right
        sorted_indices = sorted_indices[::-1]
    sorted_data.extend(row_data[sorted_indices])
hex = np.array(sorted_data)
flux = np.concatenate((bflux,rflux))

B = np.arange(4850,5045,1)  # wavelengths to evaluate at
counts=[]
for i in range(len(flux)):
    count = []
    for b in B:
        count.append(flux[i][np.where(bwave>b)[0][0]])
    counts.append(count)
counts = np.array(counts)  # .shape is (flux,wave)

function used to animate

In [ ]:
r = 0.335
cmap = matplotlib.colormaps['Greys']

def map_animation(frame):
    ax.clear()
    b = B[frame]
    norm = Normalize(vmin=np.percentile(counts[:,frame], 2), vmax=np.percentile(counts[:,frame], 98))
    for x,y,f in zip(hex[:,0], hex[:,1], range(len(flux))):
        color = cmap(norm(counts[f][frame]))
        hexag = matplotlib.patches.RegularPolygon((x,y), numVertices=6, radius=r, facecolor=color)
        ax.add_patch(hexag)
        ax.set_xlim(np.min(hex[:,0])-r, np.max(hex[:,0])+r)
        ax.set_ylim(np.min(hex[:,1])-r, np.max(hex[:,1])+r)
        ax.set_aspect('equal'); ax.invert_xaxis(); ax.set_axis_off()
        ax.set_title(rf'$\rm Wavelength={b}$')
    return ax.patches

In [ ]:
fig,ax=plt.subplots(1,1, figsize=(3.5,3.5), layout='compressed')
map_ani = FuncAnimation(fig=fig, func=map_animation, frames=len(B), interval=75, blit=False, repeat=True, cache_frame_data=False)
plt.close(fig)
HTML(map_ani.to_jshtml())

save the animation to a .gif file

In [ ]:
writer = matplotlib.animation.PillowWriter(fps=20)
map_ani.save('map_animation.gif', writer=writer)